In [2]:
from src import data_loader as dl
file_path = 'data/U13271915_20250101_20251029.csv'
portfolio_t0 = dl.load_initial_state(file_path)

if portfolio_t0:
    print("Successfully loaded initial state:")
    print(portfolio_t0['holdings'])
    print(portfolio_t0['cash'])

Successfully loaded initial state:
{'ASML': 2.0, 'CHDVDz': 45.0, 'DGE': 56.0, 'FLIN': 192.0, 'IBKR': 3.0368, 'IUSCz': 86.0, 'IVV': 12.0, 'KGXd': 47.0, 'MC': 6.0, 'MSFT': 3.0, 'VNAd': 136.0}
{'CHF': -14909.51088, 'EUR': 23.42152542, 'GBP': 15.57, 'USD': 61.55}
